In [ ]:
knitr::opts_chunk$set(echo = TRUE)
require(tidyverse)
require(data.table)
require(reshape2)

#Install additional packages
install.packages("ggpubr")
install.packages("here")
install.packages("cowplot")
install.packages("R.utils")
require(ggpubr)
require(here)
require(cowplot)
require(R.utils)


In [ ]:
#Project generated
knitr::opts_chunk$set(echo = TRUE)
dd_genes <- fread(here('data/discordant_consensus_novel_DD_associated_genes_kaplanis2020.txt'),header=FALSE,data.table=TRUE)[,V1]
hotspot_genes <- fread(here('data/hotspot_genes.txt'),header=FALSE,data.table=TRUE)[,V1]
novel_hotspot_genes <- fread(here('data/proposed_novel_hotspot_genes.txt'),header=FALSE,data.table=TRUE)[,V1]
ion_channel_genes <- fread(here('data/All_genes_with_PF00520_domain.txt'),header=FALSE,data.table=TRUE)[,V1]
hotspot_genetic_positions <- fread(here('data/hotspot_to_genomic_positions_PF00520.txt',header=TRUE,data.table=TRUE))

#External datasets
gene_tpm <- fread(here('data/external/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz'),header=TRUE,data.table=TRUE)
constraint_in <- fread(here('data/external/gnomad.v2.1.1.lof_metrics.by_gene.txt'),header=TRUE,data.table=TRUE)
mpc_in <- fread(here('data/external/mpc_missense_constraint_multiple_regions_samocha17.txt'),header=TRUE,data.table=TRUE)
ddg2p_in <- fread(here('data/external/DDG2P_22-4-2021.csv'),header=TRUE,data.table=TRUE)
omim_in <- fread(here('data/external/genemap2.txt'),header=TRUE,data.table=TRUE)


### Expression input data and counts



In [ ]:
#Remove novel hotspot genes from hotspot gene list
hotspot_genes <- hotspot_genes[!hotspot_genes %in% novel_hotspot_genes]

#Remove hotspot genes from DD consensus gene list
dd_genes <- dd_genes[!dd_genes %in% hotspot_genes]

#Parse gene TPM file
gene_tpm <- gene_tpm[,Gene_ID:=gsub("\\..*","",Name)]
gene_tpm <- data.table(melt(gene_tpm,id.vars = c('Description','Name','Gene_ID')))
colnames(gene_tpm) <- c('hgnc','ensembl_versioned','ensembl','tissue','tpm')

gene_tpm <- gene_tpm[,class:=ifelse(hgnc %in% novel_hotspot_genes,'Proposed Novel Hotspot Genes',
                                    ifelse(hgnc %in% hotspot_genes,'Hotspot Genes',
                                           ifelse(hgnc %in% dd_genes,'NDD-Associated Genes','Control Genes')))]

gene_tpm <- gene_tpm[,contains_ion_channel:=ifelse(hgnc %in% ion_channel_genes,TRUE,FALSE)]

head(gene_tpm)

gene_tpm_tab <- gene_tpm %>%
  group_by(class) %>%
  dplyr::select('hgnc','class') %>%
  unique() %>%
  count()

colnames(gene_tpm_tab) <- c('Class','Count')
  
#Table of counts by class (Control Genes, DD Consensus Genes, Hotspot Genes, Proposed Novel Hotspot Genes)
knitr::kable(gene_tpm_tab)


### Figure 5A: Tissue expression of hotspot genes compared to control and other NDD genes



In [ ]:
#Set seed
set.seed(1255)

#Randomly sample NDD-associated and control Genes
sampleGenes <- function( iterations, sampling_size ) {

  out <- vector('list',length=iterations)
  
  for( i in 1:iterations ) {
    
    sampled_dat <- gene_tpm %>% 
    filter(class!='Hotspot Genes' & class!='Proposed Novel Hotspot Genes') %>%
    group_by(tissue,class) %>% 
    sample_n(sampling_size,replace=TRUE) %>%
    mutate(i=factor(i))
    
    out[[i]] <- data.table(sampled_dat)
    
  }
  
  out_dt <- do.call(rbind,out)
  return(out_dt)

}

hotspot_tpm <- gene_tpm %>%
  filter(class=='Hotspot Genes') %>%
  mutate(i=0)

#Bind sampled genes to hotspot genes; calculate the proportion expressed, mean, and standard deviation across sets
fig_5a <- rbind(sampleGenes(1000,19),hotspot_tpm) %>%
  group_by(tissue,class,i) %>%
  summarise(
    prop_expressed = sum(tpm > 1, na.rm=T) / n(),
    .groups = 'drop_last'
  ) %>%
  ungroup() %>%
  group_by(tissue,class) %>%
  summarise(
    mean = mean(prop_expressed),
    min = mean - sd(prop_expressed),
    max = mean + sd(prop_expressed),
    .groups = 'keep'
  ) %>%
  data.table()

#Figure 5A
a_fill <- c('Control Genes' = 'skyblue3',
            'NDD-Associated Genes' = 'darkgreen',
            'Hotspot Genes' = 'darkgoldenrod')

a <- ggplot(fig_5a[class!='Proposed Novel Hotspot Genes' & class!='Hotspot Genes'],aes(x=tissue,y=mean,colour=class,ymin=min,ymax=max)) +
  geom_point(alpha=0.7,size=3,shape=15) +
  geom_linerange(alpha=0.5,size=3) + 
  geom_point(data=fig_a[class=='Hotspot Genes'],aes(x=tissue,y=mean,colour=class),size=5) +
  theme_pubr() +
  theme(axis.text = element_text(size=14), 
        axis.title = element_text(size=16), 
        axis.text.x = element_text(angle = 90, 
                                   vjust = 0.5, 
                                   hjust=1),
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_colour_manual(values=a_fill,
                      guide=guide_legend(override.aes = list(linetype = c(1,0,1),
                                                           shape = c(15,19,15),
                                                           size = c(3,5,3)))) +
  ylim(0,1) +
  xlab('') +
  ylab('Proportion expressed genes (TPM > 1)')

plot_grid(a,labels='AUTO',label_size=16)
ggsave('~/results/main/Figure_5A.svg',plot=last_plot(),device='svg',dpi=300,width=14,height=10)


### Figure 5B: TPM distribution in brain and other tissues varies across gene sets



In [ ]:
#Compute median TPM in brain and other tissues
median_gene_tpm <- gene_tpm %>%
  mutate(tissue_group = ifelse(tissue %in% brain_tissues,'Brain','Other Tissues')) %>%
  group_by(ensembl_versioned,tissue_group) %>%
  mutate(median_tpm_tissuegroup = median(tpm)) %>%
  dplyr::select(-c('tpm','tissue')) %>%
  unique() %>%
  reshape2::dcast(formula=hgnc + class + ensembl_versioned + contains_ion_channel ~ tissue_group, value.var='median_tpm_tissuegroup') %>%
  mutate(class = ifelse(hgnc %in% ion_channel_genes & class=='NDD-Associated Genes','PF00520-Containing NDD-Associated Genes',
                        ifelse(hgnc %in% ion_channel_genes & class=='Control Genes','PF00520-Containing Control Genes',class))) %>%
  data.table()

#Figure 5B
labels_b = c('NDD-Associated Genes' = 'NDD-Associated Genes',
           'PF00520-Containing NDD-Associated Genes' = 'PF00520 Domain-Containing\nNDD-Associated Genes',
           'Hotspot Genes' = 'Hotspot Genes',
           'Proposed Novel Hotspot Genes' = 'Proposed Novel Hotspot Genes')

b <- ggplot(median_gene_tpm,aes(x=log10(`Other Tissues`),
                                y=log10(Brain),
                                colour=factor(class,levels=c('Control Genes',
                                                             'NDD-Associated Genes',
                                                             'PF00520-Containing NDD-Associated Genes',
                                                             'Hotspot Genes',
                                                             'Proposed Novel Hotspot Genes')))) +
  geom_abline(slope=1,intercept=0,linetype='dashed',colour='black') +
  geom_density2d(data=median_gene_tpm[class=='Control Genes' & Brain!=0 & `Other Tissues`!=0],alpha=0.7) +
  geom_density2d(data=median_gene_tpm[class=='NDD-Associated Genes' & Brain!=0 & `Other Tissues`!=0],alpha=0.7) +
  geom_point(data=median_gene_tpm[class=='PF00520-Containing NDD-Associated Genes'],size=4) +
  geom_point(data=median_gene_tpm[class=='Hotspot Genes'],size=4) +
  geom_text(data=median_gene_tpm[class=='Proposed Novel Hotspot Genes'],
            aes(label=hgnc),size=5,fontface='italic',show.legend=FALSE) +
  theme_pubr() +
  theme(axis.text = element_text(size=14), 
        axis.title = element_text(size=16), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14),
        legend.position = c(0.75,0.13)) +
  scale_colour_manual(labels=labels_b,values=c('skyblue3',
                                               'darkgoldenrod',
                                               'darkgreen',
                                               'grey40',
                                               'tomato4'),
                      guide=guide_legend(override.aes = list(linetype=c(1,0,1,0,0),
                                                             shape=c(NA,19,NA,19,65)),
                                         nrow=5,
                                         by_row=TRUE)) +
  ylab('Log10 Median Expression in Brain Tissues') + 
  xlab('Log10 Median Expression in Other Tissues') +
  coord_cartesian(xlim=c(-3,3),ylim=c(-3,3))

plot_grid(b,labels='B',label_size=16)

ggsave('~/results/main/Figure_5B.svg',plot=last_plot(),device='svg',dpi=300,width=14,height=10)

plot_grid(a,b,labels='AUTO',label_size=16,nrow=1,rel_widths = c(1.7,1),align='hv',axis='t')

ggsave('~/results/main/Figure_5.png',plot=last_plot(),device='png',dpi=300,width=20,height=9)
ggsave('~/results/main/Figure_5.svg',plot=last_plot(),device='svg',dpi=300,width=20,height=9)


### Figure 4A: Constraint in hotspot and proposed novel hotspot genes



In [ ]:
#Get relevant constraint information
probabilistic_constraint <- c('pLI','pRec','pNull')
upper_bound_fraction <- c('oe_lof_upper','oe_mis_upper','oe_syn_upper')
observed_expected <- c('oe_lof','oe_mis','oe_syn','oe_lof_lower','oe_mis_lower','oe_syn_lower',upper_bound_fraction)
z_scores <- c('lof_z','mis_z','syn_z')
all_constraint <- unique(c(probabilistic_constraint,upper_bound_fraction,observed_expected,z_scores))

constraint <- constraint_in %>%
  subset(select=c('gene',all_constraint)) %>%
  reshape2::melt(id.vars = 'gene', value.name = 'value', var.name = 'constraint_metric') %>%
  filter(!is.na(value)) %>%
  mutate(class=ifelse(gene %in% novel_hotspot_genes,'Proposed Novel Hotspot Genes',
                      ifelse(gene %in% hotspot_genes,'Hotspot Genes',
                             ifelse(gene %in% dd_genes,'NDD-Associated Genes','Control Genes'))))

fig_4a <- constraint %>%
  filter(variable %in% observed_expected) %>%
  group_by(class,variable) %>%
  mutate(variable=gsub('oe_','',variable)) %>%
  mutate(type=substr(variable,1,3)) %>%
  mutate(bound=ifelse(grepl('upper',variable),'upper',
                      ifelse(grepl('lower',variable),'lower',''))) %>%
  group_by(class,type,bound) %>%
  summarise(
    mean = mean(value),
    .groups = 'drop_last'
  ) %>%
  mutate(bound=ifelse(bound=='','oe',bound)) %>%
  reshape2::dcast(formula=class + type ~ bound, value.var='mean')

#Figure 4A
a_labels = c('Loss of function','Missense','Synonymous')
a_levels = c('Control Genes','NDD-Associated Genes','Hotspot Genes','Proposed Novel Hotspot Genes')
a_x_labels = c('Control Genes\nn = 18648','NDD-Associated\nGenes\nn = 985','Hotspot Genes\nn = 19','Proposed Novel\nHotspot Genes\nn = 6')

a <- ggplot(fig_4a,aes(x=factor(class,levels=a_levels),y=oe,colour=type,ymin=lower,ymax=upper)) + 
  geom_pointrange(position=position_dodge(width=1),size=1.6) + 
  geom_hline(yintercept=1,linetype='dashed',colour='black') +
  theme_pubr() +
  theme(axis.text = element_text(size=14), 
        axis.title = element_text(size=16), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_x_discrete(labels=a_x_labels) +
  scale_colour_manual(labels=a_labels, values=c('tomato4','darkorange3','deeppink4')) + 
  ylab('Observed/expected counts') +
  xlab('')

ggsave('~/results/main/Figure_5A.svg',plot=last_plot(),device='svg',dpi=300,width=10,height=8)


### Figure 4B: Mutation hotspots occur in missense constrained regions within genes



In [ ]:
#Get regional missense constraint at hotspot positions
mpc <- mpc_in[,mpc_starts:=ifelse(genomic_start<genomic_end,genomic_start,genomic_end)]
mpc <- mpc[,mpc_ends:=ifelse(genomic_start<genomic_end,genomic_end,genomic_start)]
mpc <- mpc[,obs_exp:=as.numeric(sub(',','.',obs_exp))]
mpc <- mpc[,gene_name:=gene]
mpc <- mpc[,gencode_transcript:=transcript]
mpc <- mpc[,c('gene_name','gencode_transcript','mpc_starts','mpc_ends','obs_exp','region_name')]

getRegionalConstraint <- function( gene, transcript ) {
  
  mpc_temp <- mpc[gene_name==gene & gencode_transcript==transcript]
  temp_hotspots <- hotspot_genetic_positions[hgnc==gene & gencode_transcription_id==transcript]
  temp_hotspots <- temp_hotspots[,start_pos:=min(chromosome_position_base_pair_one,chromosome_position_base_pair_two,chromosome_position_base_pair_three),by=1:nrow(temp_hotspots)]
  temp_hotspots <- temp_hotspots[,end_pos:=max(chromosome_position_base_pair_one,chromosome_position_base_pair_two,chromosome_position_base_pair_three),by=1:nrow(temp_hotspots)]

  setkey(mpc_temp,mpc_starts,mpc_ends)
  setkey(temp_hotspots,start_pos,end_pos)
  
  overlaps <- foverlaps(temp_hotspots, mpc_temp, nomatch = 0)
  
  overlaps <- overlaps[,c('gene_name','gencode_transcript','obs_exp','region_name','mpc_starts','mpc_ends','start_pos','end_pos','hotspot')]
  return(overlaps)
  
}

genes <- unique(hotspot_genetic_positions[,hgnc])
regionalconstraint_out <- vector('list',length=nrow(hotspot_genetic_positions))
n = 1

for( gene in genes ) {
  
  transcripts <- unique(hotspot_genetic_positions[hgnc==gene,gencode_transcription_id])
  
  for( transcript in transcripts ) {
    
    regionalconstraint_out[[n]] <- getRegionalConstraint(gene, transcript)
    n = n + 1
    
  }
  
}

regional_constraint <- do.call(rbind,regionalconstraint_out)

regional_constraint <- regional_constraint[,class:=ifelse(gene_name %in% hotspot_genes,'Hotspot Genes',
                                    ifelse(gene_name %in% novel_hotspot_genes,'Proposed Novel Hotspot Genes',
                                           ifelse(gene_name %in% dd_genes,'NDD-Associated Genes','Control Genes')))]

colnames(regional_constraint) <- c('gene','gencode_transcript','obs_exp','region_name','mpc_starts','mpc_ends','hotspot_starts','hotspot_ends','hotspot','class')

#Get global missense constraint
global_oe_mis <- constraint %>%
  filter(variable=='oe_mis')

#Add global missense constraint to regional constraint infile and parse
fig_4b <- regional_constraint %>%
  inner_join(global_oe_mis,by='gene') %>%
  dplyr::select(obs_exp,value,gene,class.x,hotspot) %>%
  mutate(obs_exp=ifelse(obs_exp>1,1,obs_exp),
         value=ifelse(value>1,1,value)) %>%
  unique() %>%
  data.table()

#Figure 4B
b_labels <- c('Control Genes' = 'PF00520 Domain-Containing\nControl Genes\nn = 28',
              'Hotspot Genes' = 'Hotspot Genes\nn = 16',
              'NDD-Associated Genes' = 'PF00520 Domain-Containing\nNDD-Associated Genes\nn = 7')

b <- ggplot(fig_4b[class.x!='Proposed Novel Hotspot Genes'],aes(x=class.x,y=obs_exp,fill=class.x)) +
  geom_boxplot(alpha=0.8) +
  theme_pubr() +
  theme(legend.position = 'none',
        axis.text = element_text(size=14), 
        axis.title = element_text(size=16), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  ggsignif::geom_signif(
    comparisons = list(c('NDD-Associated Genes','Hotspot Genes'),
                         c('Control Genes','Hotspot Genes')),
    y_position = c(1.2,1.3),
    colour='black',
    test = 'wilcox.test') +
  scale_x_discrete(labels=b_labels) +
  scale_fill_manual(values=c('skyblue3','darkgoldenrod','darkgreen')) +
  ylab('Observed/expected regional missense constraint\nat PF00520 domains') +
  xlab('')

plot_grid(b,labels='B',label_size=16)

ggsave('~/results/main/Figure_4B.svg',plot=last_plot(),device='svg',dpi=300,width=10,height=8)

plot_grid(a,b,labels='AUTO',label_size=16,nrow=1,rel_widths = c(1.1,1),align='hv',axis='bt')

ggsave('~/results/main/Figure_4.svg',plot=last_plot(),device='svg',dpi=300,width=18,height=8)
ggsave('~/results/main/Figure_4.png',plot=last_plot(),device='png',dpi=300,width=18,height=8)


### Supplementary data



In [ ]:
#Supplementary Data S9: Mutational constraint in hotspot and proposed novel hotspot genes
supp_data_s9 <- constraint_in %>%
  subset(select=c('gene',all_constraint)) %>%
  drop_na() %>%
  mutate(class=ifelse(gene %in% novel_hotspot_genes,'Proposed Novel Hotspot Genes',
                      ifelse(gene %in% hotspot_genes,'Hotspot Genes',
                             ifelse(gene %in% dd_genes,'NDD-Associated Genes','Control Genes')))) %>%
  filter(class=='Hotspot Genes' | class=='Proposed Novel Hotspot Genes') %>%
  arrange(class,pLI) %>%
  data.table()

write.table(supp_data_s9,'~/results/supp/Supplementary_Data_S9_Constraint.txt',col.names=TRUE,
            row.names=FALSE,quote=FALSE,sep='\t')

#Supplementary Data S10: Proportion of hotspot genes expressed across tissues
prop_expressed <- gene_tpm %>%
  group_by(tissue,class) %>%
  summarise(
    num_expressed = sum(tpm > 1, na.rm=T),
    num_unexpressed = sum(tpm <= 1, na.rm=T),
    .groups = 'drop_last'
  ) %>%
  filter(class!='Proposed Novel Hotspot Genes') %>%
  data.table()

getFisher <- function( class1, class2, tissue_type ) {
  
  num_expressed_1 <- unique(prop_expressed[class==class1 & tissue==tissue_type,num_expressed])
  num_expressed_2 <- unique(prop_expressed[class==class2 & tissue==tissue_type,num_expressed])
  
  num_unexpressed_1 <- unique(prop_expressed[class==class1 & tissue==tissue_type,num_unexpressed])
  num_unexpressed_2 <- unique(prop_expressed[class==class2 & tissue==tissue_type,num_unexpressed])
  
  class1_prop <- c(num_expressed_1,num_unexpressed_1)
  class2_prop <- c(num_expressed_2,num_unexpressed_2)
  
  p <- fisher.test(cbind(class1_prop,class2_prop))$p.value
  return(p)
  
}

prop_expressed <- prop_expressed[,Fisher_p_controls:=getFisher('Hotspot Genes','Control Genes',tissue),by=tissue]
prop_expressed <- prop_expressed[,Fisher_p_NDD:=getFisher('Hotspot Genes','NDD-Associated Genes',tissue),by=tissue]
prop_expressed <- prop_expressed[,Fisher_p_controls_bonferroni:=Fisher_p_controls*54] #Bonferroni correct over 54 GTEx tissues
prop_expressed <- prop_expressed[,Fisher_p_NDD_bonferroni:=Fisher_p_NDD*54] #Bonferroni correct over 54 GTEx tissues

supp_data_s10 <- prop_expressed %>%
  mutate(Fisher_p_controls_bonferroni=ifelse(Fisher_p_controls_bonferroni>1,1,Fisher_p_controls_bonferroni),
         Fisher_p_NDD_bonferroni=ifelse(Fisher_p_NDD_bonferroni>1,1,Fisher_p_NDD_bonferroni)) %>%
  dplyr::select(class,tissue,num_expressed,num_unexpressed,Fisher_p_controls,Fisher_p_NDD,Fisher_p_controls_bonferroni,Fisher_p_NDD_bonferroni) %>%
  data.table()

colnames(supp_data_s10) <- c('Class','GTEx_Tissue','Num_Expressed Genes','Num_Unexpressed_Genes',
                             'Fisher_P_Controls','Fisher_P_NDD','Fisher_P_Controls_Bonferroni',
                             'Fisher_P_NDD_Bonferroni')

write.table(supp_data_s10,'/results/supp/Supplementary_Data_S10_Proportion_hotspot_genes_expressed_across_tissues_all.txt',
            col.names=TRUE,row.names=FALSE,quote=FALSE,sep="\t")

#Supplementary data S11: Proportion of hotspot genes expressed across tissues, PF00520 domain-containing genes
prop_expressed <- gene_tpm %>%
  filter(contains_ion_channel==TRUE) %>%
  group_by(tissue,class) %>%
  summarise(
    num_expressed = sum(tpm > 1, na.rm=T),
    num_unexpressed = sum(tpm <= 1, na.rm=T),
    .groups = 'drop_last'
  ) %>%
  filter(class!='Proposed Novel Hotspot Genes') %>%
  data.table()

prop_expressed <- prop_expressed[,Fisher_p_controls:=getFisher('Hotspot Genes','Control Genes',tissue),by=tissue]
prop_expressed <- prop_expressed[,Fisher_p_NDD:=getFisher('Hotspot Genes','NDD-Associated Genes',tissue),by=tissue]
prop_expressed <- prop_expressed[,Fisher_p_controls_bonferroni:=Fisher_p_controls*54] #Bonferroni correct over 54 GTEx tissues
prop_expressed <- prop_expressed[,Fisher_p_NDD_bonferroni:=Fisher_p_NDD*54] #Bonferroni correct over 54 GTEx tissues

supp_data_s11 <- prop_expressed %>%
  mutate(Fisher_p_controls_bonferroni=ifelse(Fisher_p_controls_bonferroni>1,1,Fisher_p_controls_bonferroni),
         Fisher_p_NDD_bonferroni=ifelse(Fisher_p_NDD_bonferroni>1,1,Fisher_p_NDD_bonferroni)) %>%
  dplyr::select(class,tissue,num_expressed,num_unexpressed,Fisher_p_controls,Fisher_p_NDD,Fisher_p_controls_bonferroni,Fisher_p_NDD_bonferroni) %>%
  data.table()

colnames(supp_data_s11) <- c('Class','GTEx_Tissue','Num_Expressed Genes','Num_Unexpressed_Genes',
                             'Fisher_P_Controls','Fisher_P_NDD','Fisher_P_Controls_Bonferroni',
                             'Fisher_P_NDD_Bonferroni')

write.table(supp_data_s11,'/results/supp/Supplementary_Data_S11_Proportion_hotspot_genes_expressed_across_tissues_PF00520.txt',
            col.names=TRUE,row.names=FALSE,quote=FALSE,sep="\t")

#Supplementary data S14: Gene sets used in analysis
ddg2p_genes <- ddg2p_in %>%
  pull(`gene symbol`) %>%
  unique()
  
omim_genes <- omim_in %>%
  separate_rows(`Gene Symbols`,sep=', ') %>%
  filter(Phenotypes!='') %>%
  pull(`Gene Symbols`) %>%
  unique()

constraint_genes <- constraint %>%
  pull(gene) %>%
  unique()

supp_data_s14 <- gene_tpm %>%
  mutate(pf00520 = contains_ion_channel) %>%
  dplyr::select(hgnc,ensembl_versioned,class,pf00520) %>%
  mutate(ddg2p=ifelse(hgnc %in% ddg2p_genes,TRUE,FALSE),
         omim=ifelse(hgnc %in% omim_genes,TRUE,FALSE),
         constraint=ifelse(hgnc %in% constraint_genes,TRUE,FALSE)) %>%
  unique() %>%
  data.table()

write.table(supp_data_s14,'/results/supp/Supplementary_Data_S14_Gene_sets_all.txt',
            col.names=TRUE,row.names=FALSE,quote=FALSE,sep="\t")

#Supplementary data S15: PF00520 domain-containing genes used in analysis
supp_data_s15 <- supp_data_s14 %>%
  filter(pf00520==TRUE) %>%
  data.table()

write.table(supp_data_s15,'/results/supp/Supplementary_Data_S15_Gene_sets_PF00520.txt',
            col.names=TRUE,row.names=FALSE,quote=FALSE,sep="\t")


### Supplementary tables



In [ ]:
#Supplementary table 4: Hotspot genes are enriched for gain-of-function mutation consequences in DDG2P
nonhap_list <- c('dominant negative','gain of function','activating','increased gene dosage')

hotspot_ddg2p_activating_genes <- length(unique(ddg2p_in[`gene symbol` %in% hotspot_genes & `mutation consequence` %in% nonhap_list,`gene symbol`]))
hotspot_ddg2p_genes <- length(unique(ddg2p_in[`gene symbol` %in% hotspot_genes,`gene symbol`])) - hotspot_ddg2p_activating_genes
ddg2p_activating_genes <- length(unique(ddg2p_in[`mutation consequence` %in% nonhap_list,`gene symbol`])) - hotspot_ddg2p_activating_genes
ddg2p_genes <- length(unique(ddg2p_in[,`gene symbol`])) - hotspot_ddg2p_genes - ddg2p_activating_genes

supp_table_4 <- rbind(c(hotspot_ddg2p_activating_genes,hotspot_ddg2p_genes),
                   c(ddg2p_activating_genes,ddg2p_genes))
colnames(supp_table_4) <- c('Gain of Function Mutation Consequence','Other Mutation Consequence')
row.names(supp_table_4) <- c('Hotspot Genes in DDG2P','Other DDG2P Genes')

knitr::kable(supp_table_4)
fisher.test(supp_table_4)

#Supplementary table 5: NDD-associated genes have higher levels of constitutive expression than control genes
constitutively_expressed_counts <- gene_tpm %>%
  group_by(ensembl_versioned) %>%
  mutate(expressed = ifelse(all(tpm < 1),'Unexpressed','Expressed')) %>%
  mutate(constitutively_expressed = ifelse(all(tpm >= 1),'Constitutively Expressed','Not Constitutively Expressed')) %>%
  ungroup() %>%
  dplyr::select(ensembl_versioned,class,expressed,constitutively_expressed) %>%
  filter(class=='Control Genes' | class=='NDD-Associated Genes') %>%
  unique() %>%
  count(class,expressed,constitutively_expressed) %>%
  group_by(class,constitutively_expressed) %>%
  summarise(n = sum(n),
            .groups = 'keep') %>%
  data.table()

supp_table_5 <- rbind(constitutively_expressed_counts[class=='Control Genes',n],
                      constitutively_expressed_counts[class=='NDD-Associated Genes',n])
colnames(supp_table_5) <- c('Constitutively Expressed','Not Constitutively Expressed')
row.names(supp_table_5) <- c('Control Genes','NDD-Associated Genes')

knitr::kable(supp_table_5)
fisher.test(supp_table_5)


### Supplementary figures



In [ ]:
#Supplementary figure 1: A significant proportion of hotspot genes have evidence of regional missense constraint compared to control and NDD-associated genes
total_genes_rmc = length(unique(mpc_in[,gene]))
dd_consensus_with_rmc = length(unique(mpc_in[gene %in% dd_genes,gene]))
hotspot_with_rmc = length(unique(mpc_in[gene %in% hotspot_genes,gene]))
novel_hotspot_with_rmc = length(unique(mpc_in[gene %in% novel_hotspot_genes,gene]))
control_genes_with_rmc = total_genes_rmc - (dd_consensus_with_rmc + hotspot_with_rmc + novel_hotspot_with_rmc)

hotspot_counts <- c(hotspot_with_rmc,length(hotspot_genes))
dd_consensus_counts <- c(dd_consensus_with_rmc,length(dd_genes))
control_counts = c(control_genes_with_rmc,length(unique(gene_tpm[class=='Control Genes',hgnc])))

supp_fig1 <- data.table(rmc_counts=c(novel_hotspot_with_rmc,hotspot_with_rmc,dd_consensus_with_rmc,control_genes_with_rmc),
                     total_counts=c(length(novel_hotspot_genes),length(hotspot_genes),length(dd_genes),length(unique(gene_tpm[class=='Control Genes',hgnc]))),
                     class=c('Proposed Novel Hotspot Genes','Hotspot Genes','NDD-Associated Genes','Control Genes'))

supp_fig1 <- supp_fig1 %>%
  mutate(prop_rmc = rmc_counts/total_counts,
         prop_other = 1 - prop_rmc) %>%
  reshape2::melt(id.vars='class') %>%
  data.table()

supp_fig1_x_labels = c('Control Genes' = 'Control Genes',
                    'Hotspot Genes' = 'Hotspot Genes',
                    'NDD-Associated Genes' = 'NDD-Associated Genes',
                    'Proposed Novel Hotspot Genes' = 'Proposed Novel\nHotspot Genes')
supp_fig1_labels = c('Genes without Evidence of Regional Missense Constraint','Genes with Evidence of Regional Missense Constraint')
supp_fig1_fill = c('darkgrey','tomato4')

ggplot(supp_c[variable=='prop_rmc' | variable=='prop_other'],
       aes(x=class,y=value,fill=factor(variable,levels=c('prop_other','prop_rmc')))) +
  geom_col() +
  theme_pubr() +
  theme(legend.position = 'top',
        axis.text = element_text(size=14), 
        axis.title = element_text(size=14), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_fill_manual(values=supp_fig1_fill,labels=supp_fig1_labels) +
  scale_x_discrete(labels=supp_fig1_x_labels) +
  xlab('') +
  ylab('Proportion of Genes')

ggsave('~/results/supp/Supplementary_Figure_1.svg',plot=last_plot(),device='svg',dpi=300,width=10,height=8)
ggsave('~/results/supp/Supplementary_Figure_1.png',plot=last_plot(),device='png',dpi=300,width=10,height=8)

fisher.test(rbind(hotspot_counts,control_counts))
fisher.test(rbind(hotspot_counts,dd_consensus_counts))
fisher.test(rbind(dd_consensus_counts,control_counts))

#Supplementary figure 2: Most mutation hotspots are more constrained for missense variation than their global missense constraint score would suggest
ggplot(fig_4b,aes(x=value,y=obs_exp,colour=class.x)) +
  geom_point(size=7) +
  geom_abline(slope=1,intercept=0,colour='black',linetype='dashed') +
  theme_pubr() +
  theme(axis.text = element_text(size=14), 
        axis.title = element_text(size=16),
        legend.position = 'top',
        legend.text = element_text(size=14),
        legend.title = element_blank()) +
  scale_colour_manual(values=c('skyblue3','darkgoldenrod','darkgreen','tomato4')) +
  xlim(0,1) +
  ylim(0,1) + 
  xlab('Global missense constraint') +
  ylab('Regional missense constraint at hotspots')

ggsave('~/results/supp/Supplementary_Figure_2.svg',plot=last_plot(),device='svg',dpi=300,width=10,height=8)
ggsave('~/results/supp/Supplementary_Figure_2.png',plot=last_plot(),device='png',dpi=300,width=10,height=8)

#Supplementary figure 3: A higher proportion of hotspot genes are expressed in brain than NDD-associated or control genes
median_gene_tpm <- gene_tpm %>%
  mutate(tissue_group = ifelse(tissue %in% brain_tissues,'Brain','Other Tissues')) %>%
  group_by(ensembl_versioned,tissue_group) %>%
  mutate(median_tpm_tissuegroup = median(tpm)) %>%
  dplyr::select(-c('tpm','tissue')) %>%
  unique() %>%
  reshape2::dcast(formula=hgnc + class + ensembl_versioned + contains_ion_channel ~ tissue_group, value.var='median_tpm_tissuegroup') %>%
  data.table()

supp_fig3 <- median_gene_tpm %>%
  group_by(class) %>%
  summarise(
    num_brain = sum(Brain > `Other Tissues`, na.rm=T),
    num_other_tissues = sum(`Other Tissues` > Brain, na.rm=T),
    total = n(),
    num_unexpressed = total - (num_brain + num_other_tissues),
    prop_brain = sum(Brain > `Other Tissues`, na.rm=T) / n(),
    prop_other = sum(`Other Tissues` > Brain, na.rm=T) / n(),
    prop_unexpressed = sum(Brain == `Other Tissues`, na.rm=T) / n(),
    .groups='drop_last'
  ) %>%
  reshape2::melt(id.vars=c('class')) %>%
  data.table()

supp_fig3_x_labels = c('Control Genes','NDD-Associated Genes','Hotspot Genes','Proposed Novel\nHotspot Genes')
supp_fig3_fill = c('prop_unexpressed' = 'darkgrey',
         'prop_other' = 'steelblue3',
         'prop_brain' = 'steelblue4')
supp_fig3_levels = c('prop_unexpressed','prop_other','prop_brain')
supp_fig3_class_levels = c('Control Genes','NDD-Associated Genes','Hotspot Genes','Proposed Novel Hotspot Genes')
supp_fig3_fill_labels = c('prop_unexpressed' = 'Unexpressed',
                'prop_other' = 'Other Tissues TPM > Brain TPM',
                'prop_brain' = 'Brain TPM > Other Tissues TPM')

ggplot(supp_fig3[variable %in% c('prop_brain','prop_other','prop_unexpressed')],
       aes(x=factor(class,levels=supp_fig3_class_levels),
           y=value,fill=factor(variable,levels=supp_fig3_levels))) + 
  geom_col(colour='black',alpha=0.8) +
  theme_pubr() +
  theme(legend.position = 'top',
        axis.text = element_text(size=14), 
        axis.title = element_text(size=14), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_fill_manual(labels=supp_fig3_fill_labels,values=supp_fig3_fill) +
  scale_x_discrete(labels=supp_fig3_x_labels) +
  xlab('') +
  ylab('Proportion of Genes')

ggsave('~/results/supp/Supplementary_Figure_3.svg',plot=last_plot(),device='svg',dpi=300,width=10,height=8)
ggsave('~/results/supp/Supplementary_Figure_3.png',plot=last_plot(),device='png',dpi=300,width=10,height=8)

hotspot_counts <- c(supp_fig3[class=='Hotspot Genes' & variable=='num_brain',value],
                    supp_fig3[class=='Hotspot Genes' & variable=='num_other_tissues',value])
dd_counts <- c(supp_fig3[class=='NDD-Associated Genes' & variable=='num_brain',value],
               supp_fig3[class=='NDD-Associated Genes' & variable=='num_other_tissues',value])
control_counts <- c(supp_fig3[class=='Control Genes' & variable=='num_brain',value],
                    supp_fig3[class=='Control Genes' & variable=='num_other_tissues',value])

fisher.test(rbind(hotspot_counts,control_counts))
fisher.test(rbind(hotspot_counts,dd_counts))

#Supplementary figure 4: Proportion of hotspot genes expressed across tissues compared to PF00520 domain-containing NDD-associated genes and PF00520 domain-containing control genes
supp_fig4 <- gene_tpm %>%
  filter(contains_ion_channel==TRUE) %>%
  filter(class!='Proposed Novel Hotspot Genes') %>%
  group_by(tissue,class) %>%
  summarise(
    prop_expressed = sum(tpm > 1, na.rm=T) / n(),
    .groups = 'keep'
  ) %>%
  data.table()

supp_fig4_colour <- c('Control Genes' = 'skyblue3',
                      'NDD-Associated Genes' = 'darkgreen',
                      'Hotspot Genes' = 'darkgoldenrod')

supp_fig4_labels <- c('Control Genes' = 'PF00520 Domain-Containing Control Genes',
                      'NDD-Associated Genes' = 'PF00520 Domain-Containing NDD-Associated Genes',
                      'Hotspot Genes' = 'Hotspot Genes')

ggplot(supp_fig4,
       aes(x=tissue,y=prop_expressed,colour=class)) +
  geom_point(size=4) +
  theme_pubr() +
  theme(axis.text = element_text(size=14), 
        axis.title = element_text(size=16), 
        axis.text.x = element_text(angle = 90, 
                                   vjust = 0.5, 
                                   hjust=1),
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_colour_manual(values=supp_fig4_colour,labels=supp_fig4_labels) +
  ylim(0,1) +
  xlab('') +
  ylab('Proportion expressed genes (TPM > 1)')

ggsave('~/results/supp/Supplementary_Figure_4.svg',plot=last_plot(),device='svg',dpi=300,width=14,height=10)
ggsave('~/results/supp/Supplementary_Figure_4.png',plot=last_plot(),device='png',dpi=300,width=14,height=10)

#Supplementary figure 5: A higher proportion of hotspot genes are expressed in brain than PF00520 domain-containing control genes
supp_fig5 <- median_gene_tpm %>%
  filter(contains_ion_channel==TRUE) %>%
  group_by(class) %>%
  summarise(
    num_brain = sum(Brain > `Other Tissues`, na.rm=T),
    num_other_tissues = sum(`Other Tissues` > Brain, na.rm=T),
    total = n(),
    num_unexpressed = total - (num_brain + num_other_tissues),
    prop_brain = sum(Brain > `Other Tissues`, na.rm=T) / n(),
    prop_other = sum(`Other Tissues` > Brain, na.rm=T) / n(),
    prop_unexpressed = sum(Brain == `Other Tissues`, na.rm=T) / n(),
    .groups='drop_last'
  ) %>%
  reshape2::melt(id.vars=c('class')) %>%
  data.table()

supp_fig5_x_labels = c('PF00520 Domain-\nContaining\nControl Genes',
                   'PF00520 Domain-\nContaining\nNDD-Associated Genes',
                   'Hotspot Genes','Proposed Novel\nHotspot Genes')
supp_fig5_fill = c('prop_unexpressed' = 'darkgrey',
         'prop_other' = 'steelblue3',
         'prop_brain' = 'steelblue4')
supp_fig5_levels = c('prop_unexpressed','prop_other','prop_brain')
supp_fig5_class_levels = c('Control Genes','NDD-Associated Genes','Hotspot Genes','Proposed Novel Hotspot Genes')
supp_fig5_fill_labels = c('prop_unexpressed' = 'Unexpressed',
                'prop_other' = 'Other Tissues TPM > Brain TPM',
                'prop_brain' = 'Brain TPM > Other Tissues TPM')

ggplot(supp_fig5[variable %in% c('prop_brain','prop_other','prop_unexpressed')],
       aes(x=factor(class,levels=supp_fig5_class_levels),
           y=value,fill=factor(variable,levels=supp_fig5_levels))) + 
  geom_col(colour='black',alpha=0.8) +
  theme_pubr() +
  theme(legend.position = 'top',
        axis.text = element_text(size=14), 
        axis.title = element_text(size=14), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_fill_manual(labels=supp_fig5_fill_labels,values=supp_fig5_fill) +
  scale_x_discrete(labels=supp_fig5_x_labels) +
  xlab('') +
  ylab('Proportion of Genes')

ggsave('~/results/supp/Supplementary_Figure_5.svg',plot=last_plot(),device='svg',dpi=300,width=10,height=8)
ggsave('~/results/supp/Supplementary_Figure_5.png',plot=last_plot(),device='png',dpi=300,width=10,height=8)

hotspot_counts_PF00520 <- c(supp_fig5[class=='Hotspot Genes' & variable=='num_brain',value],
                            supp_fig5[class=='Hotspot Genes' & variable=='num_other_tissues',value])
dd_counts_PF00520 <- c(supp_fig5[class=='NDD-Associated Genes' & variable=='num_brain',value],
                       supp_fig5[class=='NDD-Associated Genes' & variable=='num_other_tissues',value])
control_counts_PF00520 <- c(supp_fig5[class=='Control Genes' & variable=='num_brain',value],
                            supp_fig5[class=='Control Genes' & variable=='num_other_tissues',value])

fisher.test(rbind(hotspot_counts_PF00520,control_counts_PF00520))
fisher.test(rbind(hotspot_counts_PF00520,dd_counts_PF00520))

#Supplementary figure 6: TPM differences between hotspot, NDD-associated, and control genes in brain and other tissues
a <- ggplot(median_gene_tpm[class!='Proposed Novel Hotspot Genes'],
            aes(x=factor(class,levels=c('Control Genes','NDD-Associated Genes','Hotspot Genes')),y=log10(Brain),fill=class)) +
  geom_boxplot(outlier.colour = NA,alpha=0.8) +
  theme_pubr() +
  theme(axis.text = element_text(size=14),
        axis.title = element_text(size=16), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_fill_manual(values=c('skyblue3','darkgreen','darkgoldenrod')) +
  ggsignif::geom_signif(comparisons = list(c('NDD-Associated Genes','Hotspot Genes'),
                                           c('Control Genes','NDD-Associated Genes'),
                                           c('Control Genes','Hotspot Genes')),
                        map_signif_level = FALSE,
                        y_position = c(3.5,3.8,4.2),
                        colour='black') +
  xlab('') +
  ylab('Log10 Brain TPM')

b <- ggplot(median_gene_tpm[class!='Proposed Novel Hotspot Genes'],
            aes(x=factor(class,levels=c('Control Genes','NDD-Associated Genes','Hotspot Genes')),y=log10(`Other Tissues`),fill=class)) +
  geom_boxplot(outlier.colour = NA,alpha=0.8) +
  theme_pubr() +
  theme(axis.text = element_text(size=14),
        axis.title = element_text(size=16), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_fill_manual(values=c('skyblue3','darkgreen','darkgoldenrod')) +
  ggsignif::geom_signif(comparisons = list(c('NDD-Associated Genes','Hotspot Genes'),
                                           c('Control Genes','NDD-Associated Genes'),
                                           c('Control Genes','Hotspot Genes')),
                        map_signif_level = FALSE,
                        y_position = c(3.5,3.8,4.2),
                        colour='black') +
  xlab('') +
  ylab('Log10 Other Tissues TPM')
  
plot_grid(a,b,labels='AUTO',label_size=16,nrow=1)

ggsave('~/results/supp/Supplementary_Figure_6.svg',plot=last_plot(),device='svg',dpi=300,width=14,height=8)
ggsave('~/results/supp/Supplementary_Figure_6.png',plot=last_plot(),device='png',dpi=300,width=14,height=8)
